In [2]:
from groundedPL.codificacion import ToPropositionalLogic

parser = ToPropositionalLogic()
parser.debug = True

In [ ]:
### AQUÍ SU CÓDIGO
from groundedPL.logUtils import LogUtils
from itertools import combinations

NUM_FILAS = 8
NUM_COLUMNAS = 8

casillas = [(x,y) for x in range(NUM_FILAS) for y in range(NUM_COLUMNAS)]

def vecinos(pos: tuple[int,int]):
    vecinos = [(x,y) for x in range(pos[0] - 1, pos[0] + 2) for y in range(pos[1] - 1, pos[1] + 2)]
    vecinos = [casilla for casilla in vecinos if casilla != pos and 0 <= casilla[0] < NUM_FILAS and 0 <= casilla[1] < NUM_COLUMNAS]
    return vecinos


#Regla 1: Cuando uan casilla no tiene una mina, tiene un numero.
def regla_1():
    nums = [i for i in range(9)]
    formulas = []
    formulas_otoria = []
    for casilla in casillas:
        for num in nums:
            formula = f'NUM({casilla[0]},{casilla[1]},{num})'
            formulas_otoria.append(parser.to_nltk(formula))
        otoria = LogUtils.Otoria(formulas_otoria)
        formula = f'-MINA({casilla[0]},{casilla[1]}) = {otoria}'
        formulas.append(parser.to_nltk(formula))
    return LogUtils.Ytoria(formulas)
        
#Regla 2: Una casilla debe contener un solo numero
def regla_2():
    nums = [i for i in range(9)]
    formulas_ytoria = []
    formulas = []
    for casilla in casillas:
        x = casilla[0]
        y = casilla[1]
        for n in nums:
            formulas_ytoria = []
            for m in nums:
                if n != m:
                    formula = f'-NUM({x},{y},{m})'
                    formulas_ytoria.append(parser.to_nltk(formula))
            ytoria = LogUtils.Ytoria(formulas_ytoria)
            formula = f'NUM({x},{y},{n}) -> {ytoria}'
            formulas.append(parser.to_nltk(formula))
    return LogUtils.Ytoria(formulas)

            
#Regla 3: El numero de una casilla coincide con la cantidad de sus vecinos que contengan una mina.
def regla_3():
    nums = [i for i in range(9)]
    formulas_ytoria = []
    for casilla in casillas:
        vec = vecinos(casilla)
        for num in nums:
            if(num <= len(vec)):
                combinaciones = list(combinations(vec, num))
                formulas_otoria = []
                for S in combinaciones:
                    ytoria_minas = []
                    ytoria_vacia = []
                    for mina in S:
                        formula = f'MINA({mina[0]},{mina[1]})'
                        ytoria_minas.append(parser.to_nltk(formula))
                    for vacia in vec:
                        if vacia not in S:
                            formula = f'-MINA({vacia[0]},{vacia[1]})'
                            ytoria_vacia.append(parser.to_nltk(formula))
                    minas = LogUtils.Ytoria(ytoria_minas)
                    vacias = LogUtils.Ytoria(ytoria_vacia)
                    if(vacias == None):
                        formula = f'{minas}'
                    elif(minas == None):
                        formula = f'{vacias}'
                    else:
                        formula = f'{minas} & {vacias}'
                    formulas_otoria.append(parser.to_nltk(formula))
                otoria = LogUtils.Otoria(formulas_otoria)
                formula = f'NUM({casilla[0]},{casilla[1]},{num}) = {otoria}'
                formulas_ytoria.append(parser.to_nltk(formula))
    return LogUtils.Ytoria(formulas_ytoria)

#Regla 4: Si una casilla es una mina, debe estar tapada. (De modo que no acabe el juego)
def regla_4():
    formulas = []
    for casilla in casillas:
        formula = f'MINA({casilla[0]},{casilla[1]}) -> TAPADA({casilla[0]}, {casilla[1]})'
        formulas.append(parser.to_nltk(formula))
    return LogUtils.Ytoria(formulas)

#------------Reglas de elección de casilla----------

#Regla 5: Una casilla (x,y) es seleccionable si y solo si no contiene una mina
def regla_5():
    formulas = []
    for casilla in casillas:
        formula = f'ELECCION({casilla[0]},{casilla[1]}) = -MINA({casilla[0]},{casilla[1]})'
        print(formula)
        formulas.append(parser.to_nltk(formula))
    return LogUtils.Ytoria(formulas)


#Regla 6:  Si una casilla destapada (x,y) contiene al numero n, y se tienen n vecinos marcados inseleccionables 
#(es decir, se dedujo que tienen una mina), el resto de vecinos tapados no son minas.
def regla_6():
    nums = [i for i in range(9)]
    formulas_ytoria = []
    for casilla in casillas:
        vec = vecinos(casilla)
        for num in nums:
            combinaciones = list(combinations(vec, num))
            formulas_otoria = []
            for S in combinaciones:
                if num == len(vec):
                    continue
                #izquierda
                formula = f'NUM({casilla[0]},{casilla[1]},{num}) & -TAPADA({casilla[0]},{casilla[1]})'
                minas = []
                for mina in S:
                    minas.append(parser.to_nltk(f'MINA({mina[0]},{mina[1]})'))
                ytoria = LogUtils.Ytoria(minas)
                if ytoria != None:
                    formula += f' & ({ytoria})'
                formula += ' -> '
                #derecha
                vacias = []
                for vecino in vec:
                    if vecino not in S:
                        vacias.append(parser.to_nltk(f'-MINA({vecino[0]},{vecino[1]})'))
                ytoria = LogUtils.Ytoria(vacias)
                formula += f'{ytoria}'
                formulas_ytoria.append(parser.to_nltk(formula))
    return LogUtils.Ytoria(formulas_ytoria)


#Regla 7: Si una casilla destapada (x,y) contiene al numero n, y se tienen n vecinos tapados, entonces los vecinos tapados son minas.
def regla_7():
    nums = [i for i in range(9)]
    formulas_ytoria = []
    for casilla in casillas:
        vec = vecinos(casilla)
        for num in nums:
            combinaciones = list(combinations(vec, num))
            for S in combinaciones:
                if num == 0:
                    formula = f'NUM({casilla[0]},{casilla[1]},{num}) & -TAPADA({casilla[0]},{casilla[1]})'
                    minas = []
                    tapados = []
                    for vecino in vec:
                        tapados.append(f'TAPADA({vecino[0]},{vecino[1]})')
                        minas.append(f'MINA({vecino[0]},{vecino[1]})')
                    ytoria_tapados = LogUtils.Ytoria(tapados)
                    ytoria_minas = LogUtils.Ytoria(minas)
                    formula += f' & {ytoria_tapados} -> {ytoria_minas}'
                    continue
                formula = f'NUM({casilla[0]},{casilla[1]},{num}) & -TAPADA({casilla[0]},{casilla[1]})'
                tapados = []
                minas = []
                for vecino in vec:
                    if vecino in S:
                        tapados.append(parser.to_nltk(f'TAPADA({vecino[0]},{vecino[1]})'))
                        minas.append(parser.to_nltk(f'MINA({vecino[0]},{vecino[1]})'))
                    else:
                        tapados.append(parser.to_nltk(f'-TAPADA({vecino[0]},{vecino[1]})'))
                ytoria = LogUtils.Ytoria(tapados)

                if ytoria != None:
                    formula += f' & {ytoria}'
                formula += ' -> '

                ytoria = LogUtils.Ytoria(minas)
                if ytoria != None:
                    formula += f'{ytoria}'
                formulas_ytoria.append(parser.to_nltk(formula))
    return LogUtils.Ytoria(formulas_ytoria)

def reglas():
    r = []
    # print('regla1')
    # r.append(regla_1())
    # print('regla2')
    # r.append(regla_2())
    # print('regla3')
    # r.append(regla_3())
    # print('regla4')
    # r.append(regla_4())
    # print('regla5')
    # r.append(regla_5())
    # print('regla6')
    # r.append(regla_6())
    print('regla7')
    r.append(regla_7())
    print('reglas terminadas')
    return LogUtils.Ytoria(r)
reglas()

regla7
NUM(0,0,1) & -TAPADA(0,0) & (TAPADA(0,1) & -TAPADA(1,0) & -TAPADA(1,1)) -> MINA(0,1)
NUM(0,0,1) & -TAPADA(0,0) & (-TAPADA(0,1) & TAPADA(1,0) & -TAPADA(1,1)) -> MINA(1,0)
NUM(0,0,1) & -TAPADA(0,0) & (-TAPADA(0,1) & -TAPADA(1,0) & TAPADA(1,1)) -> MINA(1,1)
NUM(0,0,2) & -TAPADA(0,0) & (TAPADA(0,1) & TAPADA(1,0) & -TAPADA(1,1)) -> (MINA(0,1) & MINA(1,0))
NUM(0,0,2) & -TAPADA(0,0) & (TAPADA(0,1) & -TAPADA(1,0) & TAPADA(1,1)) -> (MINA(0,1) & MINA(1,1))
NUM(0,0,2) & -TAPADA(0,0) & (-TAPADA(0,1) & TAPADA(1,0) & TAPADA(1,1)) -> (MINA(1,0) & MINA(1,1))
NUM(0,0,3) & -TAPADA(0,0) & (TAPADA(0,1) & TAPADA(1,0) & TAPADA(1,1)) -> (MINA(0,1) & MINA(1,0) & MINA(1,1))
NUM(0,1,1) & -TAPADA(0,1) & (TAPADA(0,0) & -TAPADA(0,2) & -TAPADA(1,0) & -TAPADA(1,1) & -TAPADA(1,2)) -> MINA(0,0)
NUM(0,1,1) & -TAPADA(0,1) & (-TAPADA(0,0) & TAPADA(0,2) & -TAPADA(1,0) & -TAPADA(1,1) & -TAPADA(1,2)) -> MINA(0,2)
NUM(0,1,1) & -TAPADA(0,1) & (-TAPADA(0,0) & -TAPADA(0,2) & TAPADA(1,0) & -TAPADA(1,1) & -TAPADA(1,2)) -> 

NUM(0,1,3) & -TAPADA(0,1) & (-TAPADA(0,0) & -TAPADA(0,2) & TAPADA(1,0) & TAPADA(1,1) & TAPADA(1,2)) -> (MINA(1,0) & MINA(1,1) & MINA(1,2))
NUM(0,1,4) & -TAPADA(0,1) & (TAPADA(0,0) & TAPADA(0,2) & TAPADA(1,0) & TAPADA(1,1) & -TAPADA(1,2)) -> (MINA(0,0) & MINA(0,2) & MINA(1,0) & MINA(1,1))
NUM(0,1,4) & -TAPADA(0,1) & (TAPADA(0,0) & TAPADA(0,2) & TAPADA(1,0) & -TAPADA(1,1) & TAPADA(1,2)) -> (MINA(0,0) & MINA(0,2) & MINA(1,0) & MINA(1,2))
NUM(0,1,4) & -TAPADA(0,1) & (TAPADA(0,0) & TAPADA(0,2) & -TAPADA(1,0) & TAPADA(1,1) & TAPADA(1,2)) -> (MINA(0,0) & MINA(0,2) & MINA(1,1) & MINA(1,2))
NUM(0,1,4) & -TAPADA(0,1) & (TAPADA(0,0) & -TAPADA(0,2) & TAPADA(1,0) & TAPADA(1,1) & TAPADA(1,2)) -> (MINA(0,0) & MINA(1,0) & MINA(1,1) & MINA(1,2))
NUM(0,1,4) & -TAPADA(0,1) & (-TAPADA(0,0) & TAPADA(0,2) & TAPADA(1,0) & TAPADA(1,1) & TAPADA(1,2)) -> (MINA(0,2) & MINA(1,0) & MINA(1,1) & MINA(1,2))
NUM(0,1,5) & -TAPADA(0,1) & (TAPADA(0,0) & TAPADA(0,2) & TAPADA(1,0) & TAPADA(1,1) & TAPADA(1,2)) -> (MINA(0,0)

RecursionError: maximum recursion depth exceeded

In [ ]:
def tseitin(A):
    '''
    Algoritmo de transformacion de Tseitin
    Input: A (cadena) en notacion inorder
    Output: B (cadena), Tseitin
    '''
    # Creamos letras proposicionales nuevas
    f = inorder_to_tree(A)
    letrasp = f.letras()
    cods_letras = [ord(x) for x in letrasp]
    m = max(cods_letras) + 256
    letrasp_tseitin = [chr(x) for x in range(m, m + f.num_conec())]
    letrasp = list(letrasp) + letrasp_tseitin
    L = [] # Inicializamos lista de conjunciones
    Pila = [] # Inicializamos pila
    i = -1 # Inicializamos contador de variables nuevas
    s = A[0] # Inicializamos símbolo de trabajo
    while len(A) > 0: # Recorremos la cadena
        # print("Pila:", Pila, " L:", L, " s:", s)
        if (s in letrasp) and (len(Pila) > 0) and (Pila[-1]=='-'):
            i += 1
            atomo = letrasp_tseitin[i]
            Pila = Pila[:-1]
            Pila.append(atomo)
            L.append(atomo + "=-" + s)
            A = A[1:]
            if len(A) > 0:
                s = A[0]
        elif s == ')':
            w = Pila[-1]
            O = Pila[-2]
            v = Pila[-3]
            Pila = Pila[:len(Pila)-4]
            i += 1
            atomo = letrasp_tseitin[i]
            L.append(atomo + "=(" + v + O + w + ")")
            s = atomo
        else:
            Pila.append(s)
            A = A[1:]
            if len(A) > 0:
                s = A[0]
    if i < 0:
        atomo = Pila[-1]
    else:
        atomo = letrasp_tseitin[i]
    B = [[atomo]] + [a_clausal(x) for x in L]
    return B


In [70]:
### AQUÍ SU CÓDIGO
parser.debug = False
minas = [(2,1), (2,6), (3,1), (3,6), (7,1), (7,2), (7,7), (6,7), (5,7), (5,6), (5,5), (6,5), (7,5), (7,6)]


tablero = []
for i in range(NUM_FILAS):
    fila = []
    for j in range(NUM_COLUMNAS):
        fila.append(0)
    tablero.append(fila)

I = dict()

for tapada in minas:
    I[parser.parse(f'MINA({tapada[0]},{tapada[1]})')] = True
    I[parser.parse(f'TAPADA({tapada[0]},{tapada[1]})')] = True
    tablero[tapada[1]][tapada[0]] = '*'

    for vecino in vecinos(tapada):
        if type(tablero[vecino[1]][vecino[0]]) == int:
            tablero[vecino[1]][vecino[0]] += 1


for x in range(NUM_FILAS):
    for y in range(NUM_COLUMNAS):
        if tablero[y][x] != '*':
            num = tablero[y][x]
            I[parser.parse(f'MINA({x},{y})')] = False
            I[parser.parse(f'NUM({x},{y},{num})')] = True

for fila in tablero:
    for casilla in fila:
        print(f'{str(casilla)}', end=', ')
    print('\n')

0, 1, 2, 2, 1, 0, 1, 1, 

0, 1, *, *, 1, 0, 2, *, 

0, 1, 2, 2, 1, 0, 2, *, 

0, 0, 0, 0, 0, 0, 1, 1, 

0, 0, 0, 0, 1, 2, 3, 2, 

0, 1, 2, 2, 3, *, *, *, 

0, 1, *, *, 4, *, 8, *, 

0, 1, 2, 2, 3, *, *, *, 



In [73]:
### AQUÍ SU CÓDIGO
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualizar(I):
        # Inicializo el plano que contiene la figura
        fig, axes = plt.subplots()
        axes.get_xaxis().set_visible(False)
        axes.get_yaxis().set_visible(False)

        step = 1/NUM_FILAS
        tangulos = []
        font_size = step * 100
        colors = dict()
        colors[1] = (0,0,1)
        colors[2] = (0,0.5,0)
        colors[3] = (1,0,0)
        colors[4] = (0,0,0.5)
        colors[5] = (0.5,0,0)
        colors[6] = (0,0.5,0.5)
        colors[7] = (0,0,0)
        colors[8] = (0.5,0.5,0.5)

        # Creo los cuadrados claros en el tablero
        for atomo in I:
            if I[atomo]:
                # DECODIFICAMOS EL ÁTOMO PARA VISUALIZACIÓN
                predicado = parser.obtener_predicado(atomo)
                x, y, n = parser.obtener_argumentos(atomo)
                x = int(x)
                y = NUM_COLUMNAS - int(y) - 1
                n = int(n)
                if predicado == 'MINA':
                    tangulos.append(patches.Rectangle((step * x, step * y), step, step, facecolor='lightslategrey'))
                elif predicado == 'NUM' and n > 0:
                    plt.text(x * step + step / 2,
                             y * step + step / 2,
                             f'{n}',
                             fontsize=font_size,
                             color=colors[n])

        # Creo las líneas del tablero
        for j in range(8):
            locacion = j * step
            # Crea linea horizontal en el rectangulo
            tangulos.append(patches.Rectangle(*[(0, step + locacion), 1, 0.005],\
                    facecolor='black'))
            # Crea linea vertical en el rectangulo
            tangulos.append(patches.Rectangle(*[(step + locacion, 0), 0.005, 1],\
                    facecolor='black'))
        # Dibujo el tablero
        for t in tangulos:
            axes.add_patch(t)
        plt.show()

In [74]:
visualizar(I)